#Set up Code

In [1]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import pandasql as ps

In [3]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)

Mounted at /content/drive


# Companies Analysis and Preprocessing

In [4]:
def set_to_floats(x):
  x = str(x)
  if x == '':
    return np.nan
  if x.strip() == '-':
    return np.nan
  if x == 'nan':
    return np.nan
  return float(x.replace(',', ''))

In [5]:
companies = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CIS 550/companies.csv')
companies[' funding_total_usd '] = companies[' funding_total_usd '].apply(lambda x: set_to_floats(x))
companies

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,1750000.0,acquired,USA,NY,New York City,New York,1,2012-06-01,2012-06,2012-Q2,2012.0,2012-06-30,2012-06-30
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,4000000.0,operating,USA,CA,Los Angeles,Los Angeles,2,NaN,NaN,NaN,NaN,2010-06-04,2010-09-23
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,40000.0,operating,EST,NaN,Tallinn,Tallinn,1,2012-10-26,2012-10,2012-Q4,2012.0,2012-08-09,2012-08-09
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,1500000.0,operating,GBR,NaN,London,London,1,2011-04-01,2011-04,2011-Q2,2011.0,2011-04-01,2011-04-01
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,60000.0,operating,USA,TX,Dallas,Fort Worth,2,2014-01-01,2014-01,2014-Q1,2014.0,2014-08-17,2014-09-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49433,/organization/zzish,Zzish,http://www.zzish.com,|Analytics|Gamification|Developer APIs|iOS|And...,Education,320000.0,operating,GBR,NaN,London,London,1,2013-01-28,2013-01,2013-Q1,2013.0,2014-03-24,2014-03-24
49434,/organization/zznode-science-and-technology-co...,ZZNode Science and Technology,http://www.zznode.com,|Enterprise Software|,Enterprise Software,1587301.0,operating,CHN,NaN,Beijing,Beijing,1,NaN,NaN,NaN,NaN,2012-04-01,2012-04-01
49435,/organization/zzzzapp-com,Zzzzapp Wireless ltd.,http://www.zzzzapp.com,|Web Development|Advertising|Wireless|Mobile|,Web Development,97398.0,operating,HRV,NaN,Split,Split,5,2012-05-13,2012-05,2012-Q2,2012.0,2011-11-01,2014-09-10
49436,/organization/a-list-games,[a]list games,http://www.alistgames.com,|Games|,Games,9300000.0,operating,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,2011-11-21,2011-11-21


## Summary Stats

In [6]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49438 entries, 0 to 49437
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   permalink            49438 non-null  object 
 1   name                 49437 non-null  object 
 2   homepage_url         45989 non-null  object 
 3   category_list        45477 non-null  object 
 4    market              45470 non-null  object 
 5    funding_total_usd   40978 non-null  float64
 6   status               48124 non-null  object 
 7   country_code         44165 non-null  object 
 8   state_code           30161 non-null  object 
 9   region               44165 non-null  object 
 10  city                 43322 non-null  object 
 11  funding_rounds       49438 non-null  int64  
 12  founded_at           38554 non-null  object 
 13  founded_month        38482 non-null  object 
 14  founded_quarter      38482 non-null  object 
 15  founded_year         38482 non-null 

In [7]:
companies.describe()

,funding_total_usd,funding_rounds,founded_year
count,4.097800e+04,49438.000000,38482.000000
mean,1.591297e+07,1.696205,2007.359129
std,1.685621e+08,1.294213,7.579203
min,1.000000e+00,1.000000,1902.000000
25%,3.500000e+05,1.000000,2006.000000
50%,2.000000e+06,1.000000,2010.000000
75%,1.000000e+07,2.000000,2012.000000
max,3.007950e+10,18.000000,2014.000000


In [8]:
companies.nunique()

permalink              49436
name                   49350
homepage_url           45850
category_list          16675
 market                  753
 funding_total_usd     14629
status                     3
country_code             115
state_code                61
region                  1089
city                    4188
funding_rounds            17
founded_at              3369
founded_month            420
founded_quarter          218
founded_year             103
first_funding_at        3914
last_funding_at         3657
dtype: int64

In [9]:
companies.corr()

,funding_total_usd,funding_rounds,founded_year
funding_total_usd,1.000000,0.105506,-0.071897
funding_rounds,0.105506,1.000000,-0.058314
founded_year,-0.071897,-0.058314,1.000000


## Analysis

In [10]:
column_names = ['permalink', 'name', 'homepage_url', 'category_list', ' market ', ' funding_total_usd ', 'status', 'country_code', 'state_code', 'region', 'city', 'funding_rounds', 'founded_at', 'founded_month', 'founded_quarter', 'founded_year', 'first_funding_at', 'last_funding_at']

for col in column_names:
  print('\n\n')
  print("==========================================================================")
  print(col.upper())
  print("VALUE COUNTS")
  print(companies[col].value_counts())
  print("\n")

  print("DUPLICATE COUNTS")
  print(companies.duplicated(subset=col, keep='first').sum())
  print("\n")

  print("NULL COUNTS AND PERCENTAGE")
  print(companies[col].isnull().sum())
  print(companies[col].isnull().sum()/(companies['permalink'].nunique())*100)
  print('\n')
  
  print("NULL ROWS")
  display(companies[companies[col].isnull()])




PERMALINK
VALUE COUNTS
/organization/treasure-valley-urology-services    2
/organization/prysm                               2
/organization/waywire                             1
/organization/polybona                            1
/organization/pollfish                            1
                                                 ..
/organization/game-ventures                       1
/organization/game9z                              1
/organization/gameaccount-network                 1
/organization/gameanalytics                       1
/organization/x                                   1
Name: permalink, Length: 49436, dtype: int64


DUPLICATE COUNTS
2


NULL COUNTS AND PERCENTAGE
0
0.0


NULL ROWS


,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at





NAME
VALUE COUNTS
Roost                  4
Spire                  4
Cue                    3
Compass                3
Hubbub                 3
                      ..
Game Trust             1
Game Ventures          1
Game9z                 1
GameAccount Network    1
[x+1]                  1
Name: name, Length: 49350, dtype: int64


DUPLICATE COUNTS
87


NULL COUNTS AND PERCENTAGE
1
0.0020228173800469294


NULL ROWS


,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at
28221,/organization/tell-it-in,NaN,http://tellitin10.com,|Startups|,Startups,25000.0,closed,NaN,NaN,NaN,NaN,1,2011-10-01,2011-10,2011-Q4,2011.0,2012-03-01,2012-03-01





HOMEPAGE_URL
VALUE COUNTS
http://spaceport.io              2
http://shelby.tv                 2
http://www.kuwo.cn               2
http://gui.de                    2
http://primordialgenetics.com    2
                                ..
http://www.gamecooks.net         1
http://www.game-craft.com        1
http://www.gamedigitalplc.com    1
http://game-insight.com          1
http://www.xplusone.com/         1
Name: homepage_url, Length: 45850, dtype: int64


DUPLICATE COUNTS
3587


NULL COUNTS AND PERCENTAGE
3449
6.97669714378186


NULL ROWS


...

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,4000000.0,operating,USA,CA,Los Angeles,Los Angeles,2,NaN,NaN,NaN,NaN,2010-06-04,2010-09-23
6,/organization/fox-networks,.Fox Networks,http://www.dotfox.com,|Advertising|,Advertising,4912393.0,closed,ARG,NaN,Buenos Aires,Buenos Aires,1,NaN,NaN,NaN,NaN,2007-01-16,2007-01-16
9,/organization/01games-technology,01Games Technology,http://www.01games.hk/,|Games|,Games,41250.0,operating,HKG,NaN,Hong Kong,Hong Kong,1,NaN,NaN,NaN,NaN,2014-07-01,2014-07-01
11,/organization/1-4-all,1-4 All,NaN,|Entertainment|Games|Software|,Software,NaN,operating,USA,NC,NC - Other,Connellys Springs,1,NaN,NaN,NaN,NaN,2013-04-21,2013-04-21
19,/organization/1000jobboersen-de,1000jobboersen.de,http://www.1000jobboersen.de,|Curated Web|,Curated Web,NaN,operating,DEU,NaN,Berlin,Berlin,1,NaN,NaN,NaN,NaN,2011-09-16,2011-09-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49412,/organization/zygo-corporation,Zygo Corporation,http://www.zygo.com,|Manufacturing|,Manufacturing,9000000.0,NaN,USA,CT,CT - Other,Middlefield,1,NaN,NaN,NaN,NaN,2011-10-24,2011-10-24
49422,/organization/zymr-inc-,"Zymr, Inc.",http://www.zymr.com,|Open Source|Network Security|Networking|Virtu...,Testing,NaN,operating,USA,CA,SF Bay Area,Santa Clara,1,NaN,NaN,NaN,NaN,2012-12-12,2012-12-12
49431,/organization/zyrra,Zyrra,http://www.zyrra.com,|E-Commerce|,E-Commerce,1510500.0,operating,USA,MA,Boston,Cambridge,4,NaN,NaN,NaN,NaN,2010-10-21,2012-10-18
49434,/organization/zznode-science-and-technology-co...,ZZNode Science and Technology,http://www.zznode.com,|Enterprise Software|,Enterprise Software,1587301.0,operating,CHN,NaN,Beijing,Beijing,1,NaN,NaN,NaN,NaN,2012-04-01,2012-04-01





FIRST_FUNDING_AT
VALUE COUNTS
2012-01-01    468
2013-01-01    463
2008-01-01    422
2011-01-01    392
2007-01-01    342
             ... 
2006-12-30      1
2008-07-16      1
0019-11-20      1
2006-04-09      1
2004-10-12      1
Name: first_funding_at, Length: 3914, dtype: int64


DUPLICATE COUNTS
45524


NULL COUNTS AND PERCENTAGE
0
0.0


NULL ROWS


,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at





LAST_FUNDING_AT
VALUE COUNTS
2013-01-01    387
2014-01-01    364
2012-01-01    348
2008-01-01    302
2011-01-01    272
             ... 
2005-09-03      1
2009-05-24      1
2010-10-02      1
1998-06-30      1
2008-07-13      1
Name: last_funding_at, Length: 3657, dtype: int64


DUPLICATE COUNTS
45781


NULL COUNTS AND PERCENTAGE
0
0.0


NULL ROWS


,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at


# Rounds Analysis and Preprocessing

In [11]:
rounds = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CIS 550/rounds.csv')
rounds[' raised_amount_usd '] = rounds[' raised_amount_usd '].apply(lambda x: set_to_floats(x))
rounds = rounds.drop(columns= 'Unnamed: 16')
rounds

,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/organization/waywire,#waywire,|Entertainment|Politics|Social Media|News|,News,USA,NY,New York City,New York,/funding-round/cc409188fa2b63482bd9008f682c2efa,seed,NaN,2012-06-30,2012-06,2012-Q2,2012.0,1750000.0
1,/organization/tv-communications,&TV Communications,|Games|,Games,USA,CA,Los Angeles,Los Angeles,/funding-round/59a3669a64e39360c2b939300bcda162,venture,NaN,2010-09-23,2010-09,2010-Q3,2010.0,3000000.0
2,/organization/tv-communications,&TV Communications,|Games|,Games,USA,CA,Los Angeles,Los Angeles,/funding-round/86d22afc65107b6941e6c43c671ecbb8,venture,NaN,2010-06-04,2010-06,2010-Q2,2010.0,1000000.0
3,/organization/rock-your-paper,'Rock' Your Paper,|Publishing|Education|,Publishing,EST,NaN,Tallinn,Tallinn,/funding-round/f06b420775f7cb6c1541a9db526534bb,seed,NaN,2012-08-09,2012-08,2012-Q3,2012.0,40000.0
4,/organization/in-touch-network,(In)Touch Network,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,GBR,NaN,London,London,/funding-round/33c3f135f05d7b734b8d7b7c8ae82647,seed,NaN,2011-04-01,2011-04,2011-Q2,2011.0,1500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83865,/organization/a-list-games,[a]list games,|Games|,Games,NaN,NaN,NaN,NaN,/funding-round/57d5459df4b227527e2edb2c1f183132,seed,NaN,2011-11-21,2011-11,2011-Q4,2011.0,9300000.0
83866,/organization/x,[x+1],|Enterprise Software|,Enterprise Software,USA,NY,New York City,New York,/funding-round/9e17c1f6579169daac605d43926e02fb,debt_financing,NaN,2013-04-04,2013-04,2013-Q2,2013.0,17000000.0
83867,/organization/x,[x+1],|Enterprise Software|,Enterprise Software,USA,NY,New York City,New York,/funding-round/ba12fd8e3d341db1c81b16282018ff7e,venture,A,2008-06-01,2008-06,2008-Q2,2008.0,16000000.0
83868,/organization/x,[x+1],|Enterprise Software|,Enterprise Software,USA,NY,New York City,New York,/funding-round/deb408bbd9f539af78ea69fb532ffa75,venture,B,2011-01-03,2011-01,2011-Q1,2011.0,10000000.0


## Summary Stats

In [12]:
rounds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83870 entries, 0 to 83869
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   company_permalink        83870 non-null  object 
 1   company_name             83869 non-null  object 
 2   company_category_list    79424 non-null  object 
 3   company_market           79417 non-null  object 
 4   company_country_code     77304 non-null  object 
 5   company_state_code       56497 non-null  object 
 6   company_region           77304 non-null  object 
 7   company_city             76223 non-null  object 
 8   funding_round_permalink  83870 non-null  object 
 9   funding_round_type       83870 non-null  object 
 10  funding_round_code       22870 non-null  object 
 11  funded_at                83870 non-null  object 
 12  funded_month             83860 non-null  object 
 13   funded_quarter          83860 non-null  object 
 14  funded_year           

In [13]:
rounds.describe()

,funded_year,raised_amount_usd
count,83860.000000,7.091500e+04
mean,2011.018173,9.176873e+06
std,2.889240,9.665408e+07
min,1921.000000,1.000000e+00
25%,2010.000000,3.611505e+05
50%,2012.000000,1.700000e+06
75%,2013.000000,6.574399e+06
max,2015.000000,2.127194e+10


In [14]:
rounds.nunique()

company_permalink          49427
company_name               49344
company_category_list      16666
company_market               753
company_country_code         115
company_state_code            61
company_region              1089
company_city                4188
funding_round_permalink    83870
funding_round_type            13
funding_round_code             8
funded_at                   4159
funded_month                 308
 funded_quarter              130
funded_year                   39
 raised_amount_usd         16582
dtype: int64

## Analysis

In [15]:
column_names = ['company_permalink', 'company_name', 'company_category_list', 'company_market', 'company_country_code', 'company_state_code', 'company_region', 'company_city', 'funding_round_permalink', 'funding_round_type', 'funding_round_code', 'funded_at', 'funded_month', ' funded_quarter ', 'funded_year', ' raised_amount_usd ']

for col in column_names:
  print('\n\n')
  print("==========================================================================")
  print(col.upper())
  print("VALUE COUNTS")
  print(rounds[col].value_counts())
  print("\n")

  print("DUPLICATE COUNTS")
  print(rounds.duplicated(subset=col, keep='first').sum())
  print("\n")

  print("NULL COUNTS AND PERCENTAGE")
  print(rounds[col].isnull().sum())
  print(rounds[col].isnull().sum()/len(rounds)*100)
  print('\n')
  
  print("NULL ROWS")
  display(rounds[rounds[col].isnull()])




COMPANY_PERMALINK
VALUE COUNTS
/organization/solarflare               18
/organization/mobivery                 16
/organization/terascala                15
/organization/aperto-networks          15
/organization/instamed                 15
                                       ..
/organization/itsalat-international     1
/organization/itsdapper                 1
/organization/itsgoinon                 1
/organization/itsmyurls                 1
/organization/waywire                   1
Name: company_permalink, Length: 49427, dtype: int64


DUPLICATE COUNTS
34443


NULL COUNTS AND PERCENTAGE
0
0.0


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd





COMPANY_NAME
VALUE COUNTS
Solarflare Communications    18
Mobivery                     16
InstaMed                     15
Aperto Networks              15
Terascala                    15
                             ..
Iframe Apps                   1
iFrat Wars                    1
IfThisFits                    1
ifussss                       1
Lost Property Heaven          1
Name: company_name, Length: 49344, dtype: int64


DUPLICATE COUNTS
34525


NULL COUNTS AND PERCENTAGE
1
0.0011923214498628831


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
47686,/organization/tell-it-in,NaN,|Startups|,Startups,NaN,NaN,NaN,NaN,/funding-round/9c987e616755a78c51a4aa67c27a2a93,seed,NaN,2012-03-01,2012-03,2012-Q1,2012.0,25000.0





COMPANY_CATEGORY_LIST
VALUE COUNTS
|Biotechnology|                                      7429
|Software|                                           5824
|Mobile|                                             2106
|E-Commerce|                                         1978
|Clean Technology|                                   1976
                                                     ... 
|Software|Databases|Android|Mobile|Games|               1
|Health Care|Web Tools|Software|                        1
|Parenting|Social Media|Kids|Health and Wellness|       1
|Internet|Nonprofits|Software|                          1
|Entertainment|Politics|Social Media|News|              1
Name: company_category_list, Length: 16666, dtype: int64


DUPLICATE COUNTS
67203


NULL COUNTS AND PERCENTAGE
4446
5.301061166090379


NULL ROWS


...

,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
60,/organization/12-labs,12 Labs,NaN,NaN,NaN,NaN,NaN,NaN,/funding-round/958803df0c565d11afba52347f7efcde,seed,NaN,0026-11-14,NaN,NaN,NaN,765000.0
2637,/organization/agflow,AgFlow,|Software|,Software,CHE,NaN,Geneva,Geneva,/funding-round/a4fcc1845670f82284af88690d1db2ad,seed,NaN,0020-06-14,NaN,NaN,NaN,NaN
11563,/organization/buru-buru,Buru Buru,|Startups|Internet|Retail|Design|Art|E-Commerce|,Startups,ITA,NaN,Firenze,Firenze,/funding-round/1e22bfd0485bd84bd78a880edd8d25a6,seed,NaN,0019-11-20,NaN,NaN,NaN,NaN
24877,/organization/exploco,Exploco,|Adventure Travel|,Adventure Travel,AUS,NaN,Perth,Perth,/funding-round/f5769fb890cd1ab33f70abf959d932c9,seed,NaN,0201-01-01,NaN,NaN,NaN,NaN
50199,/organization/nubank,Nubank,|Consumer Internet|Financial Services|,Financial Services,BRA,NaN,Sao Paulo,São Paulo,/funding-round/6aaddf226adfaa6eb7dc8497331d9a7b,seed,NaN,0007-05-13,NaN,NaN,NaN,2000000.0
54020,/organization/peoplegoal,PeopleGoal,|Enterprise Software|,Enterprise Software,NaN,NaN,NaN,NaN,/funding-round/c0990da936fc935fb4eb4dc4c9c8d129,seed,NaN,0001-05-14,NaN,NaN,NaN,NaN
61572,/organization/rotor,Rotor,|Developer Tools|Music|Video|,Video,IRL,NaN,Dublin,Dublin,/funding-round/4f2416bb4db59e5e585ce3d57cd906ef,convertible_note,NaN,0029-09-14,NaN,NaN,NaN,NaN
63245,/organization/securenet-payment-systems,SecureNet Payment Systems,|Trading|Mobile Payments|Payments|E-Commerce|,Payments,USA,TX,Austin,Austin,/funding-round/d72b60bd12b826d684820c6d28957899,private_equity,NaN,0011-11-14,NaN,NaN,NaN,NaN
64605,/organization/shopboostr,Shopboostr,|SaaS|Reviews and Recommendations|E-Commerce|,SaaS,NaN,NaN,NaN,NaN,/funding-round/52b8b119eab4ea1af360c5306526c6a9,seed,NaN,0001-11-14,NaN,NaN,NaN,NaN
72904,/organization/the-urban-roosters,The Urban Roosters,NaN,NaN,NaN,NaN,NaN,NaN,/funding-round/45a194fa4e0eec9cfd60fa00a3c098cc,angel,NaN,0001-07-14,NaN,NaN,NaN,NaN





FUNDED_YEAR
VALUE COUNTS
2013.0    15969
2014.0    15825
2012.0    12512
2011.0    10471
2010.0     8647
2009.0     6515
2008.0     4235
2007.0     3535
2006.0     2640
2005.0     1753
2004.0      414
2000.0      266
2003.0      259
2002.0      203
2001.0      191
1999.0      176
1998.0       61
1996.0       32
1997.0       30
1995.0       21
1994.0       15
1993.0       15
1990.0       14
1992.0       12
1991.0       10
1985.0        6
1987.0        6
1986.0        4
1989.0        4
1984.0        4
1960.0        3
1982.0        3
1974.0        2
1988.0        2
1979.0        1
1973.0        1
1983.0        1
2015.0        1
1921.0        1
Name: funded_year, dtype: int64


DUPLICATE COUNTS
83830


NULL COUNTS AND PERCENTAGE
10
0.01192321449862883


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
60,/organization/12-labs,12 Labs,NaN,NaN,NaN,NaN,NaN,NaN,/funding-round/958803df0c565d11afba52347f7efcde,seed,NaN,0026-11-14,NaN,NaN,NaN,765000.0
2637,/organization/agflow,AgFlow,|Software|,Software,CHE,NaN,Geneva,Geneva,/funding-round/a4fcc1845670f82284af88690d1db2ad,seed,NaN,0020-06-14,NaN,NaN,NaN,NaN
11563,/organization/buru-buru,Buru Buru,|Startups|Internet|Retail|Design|Art|E-Commerce|,Startups,ITA,NaN,Firenze,Firenze,/funding-round/1e22bfd0485bd84bd78a880edd8d25a6,seed,NaN,0019-11-20,NaN,NaN,NaN,NaN
24877,/organization/exploco,Exploco,|Adventure Travel|,Adventure Travel,AUS,NaN,Perth,Perth,/funding-round/f5769fb890cd1ab33f70abf959d932c9,seed,NaN,0201-01-01,NaN,NaN,NaN,NaN
50199,/organization/nubank,Nubank,|Consumer Internet|Financial Services|,Financial Services,BRA,NaN,Sao Paulo,São Paulo,/funding-round/6aaddf226adfaa6eb7dc8497331d9a7b,seed,NaN,0007-05-13,NaN,NaN,NaN,2000000.0
54020,/organization/peoplegoal,PeopleGoal,|Enterprise Software|,Enterprise Software,NaN,NaN,NaN,NaN,/funding-round/c0990da936fc935fb4eb4dc4c9c8d129,seed,NaN,0001-05-14,NaN,NaN,NaN,NaN
61572,/organization/rotor,Rotor,|Developer Tools|Music|Video|,Video,IRL,NaN,Dublin,Dublin,/funding-round/4f2416bb4db59e5e585ce3d57cd906ef,convertible_note,NaN,0029-09-14,NaN,NaN,NaN,NaN
63245,/organization/securenet-payment-systems,SecureNet Payment Systems,|Trading|Mobile Payments|Payments|E-Commerce|,Payments,USA,TX,Austin,Austin,/funding-round/d72b60bd12b826d684820c6d28957899,private_equity,NaN,0011-11-14,NaN,NaN,NaN,NaN
64605,/organization/shopboostr,Shopboostr,|SaaS|Reviews and Recommendations|E-Commerce|,SaaS,NaN,NaN,NaN,NaN,/funding-round/52b8b119eab4ea1af360c5306526c6a9,seed,NaN,0001-11-14,NaN,NaN,NaN,NaN
72904,/organization/the-urban-roosters,The Urban Roosters,NaN,NaN,NaN,NaN,NaN,NaN,/funding-round/45a194fa4e0eec9cfd60fa00a3c098cc,angel,NaN,0001-07-14,NaN,NaN,NaN,NaN





 RAISED_AMOUNT_USD 
VALUE COUNTS
1000000.0     2267
2000000.0     1691
500000.0      1660
5000000.0     1585
10000000.0    1569
              ... 
4891000.0        1
8676000.0        1
2874000.0        1
7483000.0        1
12955.0          1
Name:  raised_amount_usd , Length: 16582, dtype: int64


DUPLICATE COUNTS
67287


NULL COUNTS AND PERCENTAGE
12955
15.446524382973651


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
10,/organization/004-technologies,004 Technologies,|Software|,Software,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,2014-07-24,2014-07,2014-Q3,2014.0,NaN
13,/organization/1-4-all,1-4 All,|Entertainment|Games|Software|,Software,USA,NC,NC - Other,Connellys Springs,/funding-round/e97a192e13ea0ee3c4f71136b4f3ec16,equity_crowdfunding,NaN,2013-04-21,2013-04,2013-Q2,2013.0,NaN
14,/organization/1-800-dentist,1-800-DENTIST,|Health and Wellness|,Health and Wellness,USA,CA,Los Angeles,Los Angeles,/funding-round/5274aacc211163fc7c86539ce94bbacc,undisclosed,NaN,2010-08-19,2010-08,2010-Q3,2010.0,NaN
16,/organization/1-618-technology,1.618 Technology,|Real Estate|,Real Estate,USA,FL,Orlando,Orlando,/funding-round/83b8f4c7d37ecef5e001a5e953bf461a,equity_crowdfunding,NaN,2014-01-22,2014-01,2014-Q1,2014.0,NaN
25,/organization/1000jobboersen-de,1000jobboersen.de,|Curated Web|,Curated Web,DEU,NaN,Berlin,Berlin,/funding-round/3407d447c84490721358e5c0eb23e3a1,venture,B,2011-09-16,2011-09,2011-Q3,2011.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83816,/organization/zymergen,Zymergen,|Biotechnology|,Biotechnology,USA,CA,SF Bay Area,San Francisco,/funding-round/6b08eaf89f9a6211f1c83494c2bf8e82,seed,NaN,2014-01-01,2014-01,2014-Q1,2014.0,NaN
83825,/organization/zympi,Zympi,|Networking|Web Hosting|Mobile|,Web Hosting,NaN,NaN,NaN,NaN,/funding-round/4aaf79fabd66416d679c9830c01f31d1,equity_crowdfunding,NaN,2011-11-22,2011-11,2011-Q4,2011.0,NaN
83826,/organization/zymr-inc-,"Zymr, Inc.",|Open Source|Network Security|Networking|Virtu...,Testing,USA,CA,SF Bay Area,Santa Clara,/funding-round/9048d1a80edeae803ed1790c9f473eec,angel,NaN,2012-12-12,2012-12,2012-Q4,2012.0,NaN
83827,/organization/zyncd,Zyncd,|Knowledge Management|Crowdsourcing|,Crowdsourcing,GBR,NaN,London,London,/funding-round/5a0d321abcfef61a8bae383176ac46e7,seed,NaN,2014-02-01,2014-02,2014-Q1,2014.0,NaN


#Investments Analysis and Preprocessing

In [16]:
investments = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CIS 550/investments.csv')
investments['raised_amount_usd'] = investments['raised_amount_usd'].apply(lambda x: set_to_floats(x))
investments

,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,...,investor_region,investor_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/organization/test-company-3,test company,NaN,NaN,ARE,NaN,Dubai,Dubai,/person/jamessss-bondddd,jamessss bondddd,...,NaN,NaN,/funding-round/c308019016ead7afb2a1d117018eb6fc,seed,NaN,1921-09-01,1921-09,1921-Q3,1921,1000.0
1,/organization/andrewburnett-com-ltd,AndrewBurnett.com Ltd,|Internet|SEO|Services|Public Relations|Social...,Internet,GBR,NaN,Edinburgh,Edinburgh,/organization/ekaf,Ekaf,...,NaN,NaN,/funding-round/14fe2864e02d0f15ddc3ec8eacdc8e1b,seed,NaN,1974-01-01,1974-01,1974-Q1,1974,NaN
2,/organization/abo-data,ABO Data,|Enterprise Software|,Enterprise Software,USA,TX,TX - Other,Italy,/person/antonio-murroni,ANTONIO MURRONI,...,NaN,NaN,/funding-round/809e211b969c3f66440fc15ffcd29385,seed,NaN,1979-01-01,1979-01,1979-Q1,1979,1000000.0
3,/organization/abo-data,ABO Data,|Enterprise Software|,Enterprise Software,USA,TX,TX - Other,Italy,/person/filippo-murroni,FILIPPO Murroni,...,NaN,NaN,/funding-round/809e211b969c3f66440fc15ffcd29385,seed,NaN,1979-01-01,1979-01,1979-Q1,1979,1000000.0
4,/organization/ikro,Ikro,NaN,NaN,BRA,NaN,BRA - Other,Canoas,/organization/crp-companhia-de-participacoes,CRP Companhia de Participações,...,NaN,NaN,/funding-round/46c353a8249170cc4b6ab89a522fefdc,venture,A,1982-06-01,1982-06,1982-Q2,1982,724000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114501,/organization/helpling,Helpling,|Consumers|,Consumers,DEU,NaN,Berlin,Berlin,/person/lukasz-gadowski,Lukasz Gadowski,...,NaN,NaN,/funding-round/681af1b330a13043b17aeac45d841b88,venture,A,2014-12-02,2014-12,2014-Q4,2014,17000000.0
114502,/organization/appknox,Appknox,|Mobile Security|SaaS|Mobile|,Mobile Security,SGP,NaN,Singapore,Singapore,/person/rajan-anandan,Rajan Anandan,...,NaN,NaN,/funding-round/bfb0b5859be2431f35882bc03370dd67,venture,NaN,2014-12-02,2014-12,2014-Q4,2014,NaN
114503,/organization/strap,Strap,|Apps|,Apps,USA,OH,Cincinnati,Cincinnati,/person/trevor-wright,Trevor Wright,...,NaN,NaN,/funding-round/375e6c61fec5b3854ecbe5732dfdcabe,seed,NaN,2014-12-02,2014-12,2014-Q4,2014,1200000.0
114504,/organization/strap,Strap,|Apps|,Apps,USA,OH,Cincinnati,Cincinnati,/person/wendy-lea,Wendy Lea,...,NaN,NaN,/funding-round/375e6c61fec5b3854ecbe5732dfdcabe,seed,NaN,2014-12-02,2014-12,2014-Q4,2014,1200000.0


##Summary Stats

In [17]:
investments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114506 entries, 0 to 114505
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   company_permalink        114506 non-null  object 
 1   company_name             114505 non-null  object 
 2   company_category_list    111242 non-null  object 
 3   company_market           111240 non-null  object 
 4   company_country_code     107147 non-null  object 
 5   company_state_code       79158 non-null   object 
 6   company_region           107147 non-null  object 
 7   company_city             105801 non-null  object 
 8   investor_permalink       114440 non-null  object 
 9   investor_name            114440 non-null  object 
 10  investor_category_list   30507 non-null   object 
 11  investor_market          30455 non-null   object 
 12  investor_country_code    86521 non-null   object 
 13  investor_state_code      62274 non-null   object 
 14  inve

In [18]:
investments.describe()

,funded_year,raised_amount_usd
count,114506.000000,1.010910e+05
mean,2010.686174,1.267140e+07
std,3.065592,4.725429e+07
min,1921.000000,1.400000e+01
25%,2009.000000,1.200000e+06
50%,2012.000000,4.600000e+06
75%,2013.000000,1.300000e+07
max,2014.000000,5.800000e+09


In [19]:
investments.nunique()

company_permalink          32335
company_name               32284
company_category_list      11874
company_market               712
company_country_code         101
company_state_code            61
company_region               965
company_city                2901
investor_permalink         22276
investor_name              22037
investor_category_list      1160
investor_market              272
investor_country_code         85
investor_state_code           59
investor_region              589
investor_city               1381
funding_round_permalink    50280
funding_round_type            13
funding_round_code             8
funded_at                   3923
funded_month                 300
funded_quarter               123
funded_year                   36
raised_amount_usd           7452
dtype: int64

##Analysis

In [20]:
column_names = ['company_permalink', 'company_name', 'company_category_list', 'company_market', 'company_country_code', 'company_state_code', 'company_region', 'company_city', 'investor_permalink', 'investor_name', 'investor_category_list', 'investor_market', 'investor_country_code','investor_state_code', 'investor_region', 'investor_city', 'funding_round_permalink', 'funding_round_type', 'funding_round_code', 'funded_at', 'funded_month', 'funded_quarter', 'funded_year', 'raised_amount_usd']

for col in column_names:
  print('\n\n')
  print("==========================================================================")
  print(col.upper())
  print("VALUE COUNTS")
  print(investments[col].value_counts())
  print("\n")

  print("DUPLICATE COUNTS")
  print(investments.duplicated(subset=col, keep='first').sum())
  print("\n")

  print("NULL COUNTS AND PERCENTAGE")
  print(investments[col].isnull().sum())
  print(investments[col].isnull().sum()/len(investments)*100)
  print('\n')
  
  print("NULL ROWS")
  display(investments[investments[col].isnull()])




COMPANY_PERMALINK
VALUE COUNTS
/organization/fab-com            60
/organization/ecomom             59
/organization/cardiodx           57
/organization/practice-fusion    55
/organization/path               53
                                 ..
/organization/reaqua-systems      1
/organization/bindhq              1
/organization/godigex             1
/organization/dimers-lab          1
/organization/vonjour-com         1
Name: company_permalink, Length: 32335, dtype: int64


DUPLICATE COUNTS
82171


NULL COUNTS AND PERCENTAGE
0
0.0


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,...,investor_region,investor_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd





COMPANY_NAME
VALUE COUNTS
Fab                60
ecomom             59
CardioDx           57
Practice Fusion    55
Path               53
                   ..
Startlocal          1
Game Cooks          1
Yoink Games         1
Zeto                1
Vonjour             1
Name: company_name, Length: 32284, dtype: int64


DUPLICATE COUNTS
82221


NULL COUNTS AND PERCENTAGE
1
0.0008733166820952613


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,...,investor_region,investor_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
59891,/organization/tell-it-in,NaN,|Startups|,Startups,NaN,NaN,NaN,NaN,/organization/portland-seed-fund,Portland Seed Fund,...,"Portland, Oregon",Portland,/funding-round/9c987e616755a78c51a4aa67c27a2a93,seed,NaN,2012-03-01,2012-03,2012-Q1,2012,25000.0





COMPANY_CATEGORY_LIST
VALUE COUNTS
|Biotechnology|                                                      7762
|Software|                                                           7224
|Mobile|                                                             2979
|Enterprise Software|                                                2710
|E-Commerce|                                                         2616
                                                                     ... 
|Consumer Goods|Beauty|Health Care|                                     1
|Security|Lighting|Semiconductors|Clean Technology|                     1
|Discounts|Location Based Services|Mobile|Messaging|SMS|Software|       1
|Hospitality|Travel|                                                    1
|Services|Wireless|Telecommunications|                                  1
Name: company_category_list, Length: 11874, dtype: int64


DUPLICATE COUNTS
102631


NULL COUNTS AND PERCENTAGE
3264
2.850505650358933


NULL ROWS


...

,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,...,investor_region,investor_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd





FUNDED_YEAR
VALUE COUNTS
2013    21041
2014    20012
2012    16583
2011    13089
2010     9708
2008     7366
2009     7031
2007     6946
2006     5735
2005     4290
2004      707
2000      455
2003      374
2002      359
2001      309
1999      254
1998       65
1997       32
1996       30
1990       19
1995       17
1994       16
1993       15
1992       12
1991       10
1987        7
1985        5
1986        4
1982        3
1989        3
1988        2
1984        2
1979        2
1974        1
1983        1
1921        1
Name: funded_year, dtype: int64


DUPLICATE COUNTS
114470


NULL COUNTS AND PERCENTAGE
0
0.0


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,...,investor_region,investor_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd





RAISED_AMOUNT_USD
VALUE COUNTS
10000000.0    3385
1000000.0     3310
5000000.0     2585
2000000.0     2568
15000000.0    2229
              ... 
1118770.0        1
215049.0         1
107524.0         1
2463690.0        1
14081347.0       1
Name: raised_amount_usd, Length: 7452, dtype: int64


DUPLICATE COUNTS
107053


NULL COUNTS AND PERCENTAGE
13415
11.71554329030793


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,...,investor_region,investor_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
1,/organization/andrewburnett-com-ltd,AndrewBurnett.com Ltd,|Internet|SEO|Services|Public Relations|Social...,Internet,GBR,NaN,Edinburgh,Edinburgh,/organization/ekaf,Ekaf,...,NaN,NaN,/funding-round/14fe2864e02d0f15ddc3ec8eacdc8e1b,seed,NaN,1974-01-01,1974-01,1974-Q1,1974,NaN
8,/organization/lenovo,Lenovo,|Hardware|,Hardware,USA,NC,Raleigh,Morrisville,/organization/chinese-academy-of-sciences,Chinese Academy Of Sciences,...,Beijing,Beijing,/funding-round/0f444f3fdb0c1058aa70eeba9f0f5089,undisclosed,NaN,1984-01-01,1984-01,1984-Q1,1984,NaN
9,/organization/mixertech,Mixertech,NaN,NaN,NaN,NaN,NaN,NaN,/organization/crp-companhia-de-participacoes,CRP Companhia de Participações,...,NaN,NaN,/funding-round/4316813c5da6e06f2a15fa25c7946a73,venture,A,1984-05-01,1984-05,1984-Q2,1984,NaN
13,/organization/aehr-test-systems,Aehr Test Systems,|Hardware + Software|,Hardware + Software,USA,CA,SF Bay Area,Fremont,/organization/summit-partners,Summit Partners,...,Boston,Boston,/funding-round/3b02cf325be61412c2af1eb545f4d568,undisclosed,NaN,1985-09-19,1985-09,1985-Q3,1985,NaN
14,/organization/fsi-international,FSI International,|Semiconductors|,Semiconductors,USA,MN,Minneapolis,Chaska,/organization/summit-partners,Summit Partners,...,Boston,Boston,/funding-round/34b482e681b7e2ec7b0c7886e9f9d90d,undisclosed,NaN,1985-10-20,1985-10,1985-Q4,1985,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114360,/organization/zendrive,Zendrive,|Automotive|,Automotive,USA,CA,SF Bay Area,San Francisco,/organization/fontinalis-partners,Fontinalis Partners,...,Detroit,Detroit,/funding-round/31204ec2abf4ddf45667d231a1b11be3,venture,NaN,2014-11-26,2014-11,2014-Q4,2014,NaN
114407,/organization/orori,Orori,NaN,NaN,IDN,NaN,Jakarta,Jakarta,/organization/ideosource,Ideosource,...,Jakarta,Jakarta,/funding-round/d0f5ccae89755f0c35deaf80b6cfe4ac,seed,NaN,2014-11-28,2014-11,2014-Q4,2014,NaN
114476,/organization/appknox,Appknox,|Mobile Security|SaaS|Mobile|,Mobile Security,SGP,NaN,Singapore,Singapore,/organization/jungle-ventures,Jungle Ventures,...,Singapore,Singapore,/funding-round/bfb0b5859be2431f35882bc03370dd67,venture,NaN,2014-12-02,2014-12,2014-Q4,2014,NaN
114502,/organization/appknox,Appknox,|Mobile Security|SaaS|Mobile|,Mobile Security,SGP,NaN,Singapore,Singapore,/person/rajan-anandan,Rajan Anandan,...,NaN,NaN,/funding-round/bfb0b5859be2431f35882bc03370dd67,venture,NaN,2014-12-02,2014-12,2014-Q4,2014,NaN


#Acquisitions EDA and Preprocessing

In [21]:
acquisitions = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CIS 550/acquisitions.csv')
acquisitions[' price_amount '] = acquisitions[' price_amount '].apply(lambda x: set_to_floats(x))
acquisitions

,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,acquirer_permalink,acquirer_name,...,acquirer_country_code,acquirer_state_code,acquirer_region,acquirer_city,acquired_at,acquired_month,acquired_quarter,acquired_year,price_amount,price_currency_code
0,/organization/waywire,#waywire,|Entertainment|Politics|Social Media|News|,News,USA,NY,New York City,New York,/organization/magnify,Waywire Networks,...,USA,NY,New York City,New York,2013-10-17,2013-10,2013-Q4,2013.0,NaN,USD
1,/organization/fluff-friends,(fluff)Friends,NaN,NaN,NaN,NaN,NaN,NaN,/organization/social-gaming-network,SGN (Social Gaming Network),...,USA,CA,Los Angeles,Beverly Hills,2008-09-16,2008-09,2008-Q3,2008.0,NaN,USD
2,/organization/red,(RED),|Nonprofits|,Nonprofits,USA,NY,New York City,New York,/organization/nationstar-mortgage-holdings,Nationstar Mortgage Holdings,...,USA,TX,Dallas,Lewisville,2014-05-08,2014-05,2014-Q2,2014.0,18000000.0,USD
3,/organization/vandaele-holdings,.,NaN,NaN,NaN,NaN,NaN,NaN,/organization/hi7e,HI7E,...,USA,FL,Palm Beaches,West Palm Beach,2011-01-01,2011-01,2011-Q1,2011.0,NaN,USD
4,/organization/co-internet,.CO,|Registrars|Domains|Curated Web|,Registrars,USA,FL,Miami,Miami,/organization/neustar,Neustar,...,USA,VA,"Washington, D.C.",Sterling,2014-03-20,2014-03,2014-Q1,2014.0,109000000.0,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13064,/organization/zync,Zync,|Curated Web|,Curated Web,NaN,NaN,NaN,NaN,/organization/where-com,Where,...,USA,MA,Boston,Boston,2008-02-07,2008-02,2008-Q1,2008.0,NaN,USD
13065,/organization/zync-render,Zync Render,|Digital Media|,Digital Media,USA,MA,Boston,Boston,/organization/google,Google,...,USA,CA,SF Bay Area,Mountain View,2014-08-26,2014-08,2014-Q3,2014.0,NaN,USD
13066,/organization/zynk-mobile,Zynk Mobile,NaN,NaN,NaN,NaN,NaN,NaN,/organization/zenvia,Zenvia,...,BRA,NaN,Porto Alegre,Porto Alegre,2013-03-01,2013-03,2013-Q1,2013.0,NaN,USD
13067,/organization/zyrion,Zyrion Inc,|Networking|Software|,Software,USA,CA,SF Bay Area,Sunnyvale,/organization/kaseya,Kaseya,...,NaN,NaN,NaN,NaN,2013-06-25,2013-06,2013-Q2,2013.0,NaN,USD


##Summary Stats

In [22]:
acquisitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13069 entries, 0 to 13068
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       13069 non-null  object 
 1   company_name            13069 non-null  object 
 2   company_category_list   9786 non-null   object 
 3   company_market          9784 non-null   object 
 4   company_country_code    9782 non-null   object 
 5   company_state_code      7519 non-null   object 
 6   company_region          9783 non-null   object 
 7   company_city            9584 non-null   object 
 8   acquirer_permalink      13069 non-null  object 
 9   acquirer_name           13069 non-null  object 
 10  acquirer_category_list  11565 non-null  object 
 11  acquirer_market         11559 non-null  object 
 12  acquirer_country_code   12228 non-null  object 
 13  acquirer_state_code     9782 non-null   object 
 14  acquirer_region         12228 non-null

In [23]:
acquisitions.describe()

,acquired_year,price_amount
count,13067.000000,3.711000e+03
mean,2010.366037,7.422824e+08
std,3.695350,4.048504e+09
min,1960.000000,1.000000e+00
25%,2009.000000,1.845000e+07
50%,2011.000000,8.300000e+07
75%,2013.000000,3.427500e+08
max,2014.000000,1.500000e+11


In [24]:
acquisitions.nunique()

company_permalink         12790
company_name              12783
company_category_list      3649
company_market              536
company_country_code         73
company_state_code           59
company_region              596
company_city               1816
acquirer_permalink         6632
acquirer_name              6629
acquirer_category_list     2371
acquirer_market             451
acquirer_country_code        67
acquirer_state_code          57
acquirer_region             495
acquirer_city              1410
acquired_at                3078
acquired_month              279
acquired_quarter            111
acquired_year                37
 price_amount              1311
price_currency_code          11
dtype: int64

##Analysis

In [25]:
column_names = ['company_permalink', 'company_name', 'company_category_list', 'company_market', 'company_country_code', 'company_state_code', 'company_region', 'company_city', 'acquirer_permalink', 'acquirer_name', 'acquirer_category_list', 'acquirer_market', 'acquirer_country_code','acquirer_state_code', 'acquirer_region', 'acquirer_city', 'acquired_at', 'acquired_month', 'acquired_quarter', 'acquired_year', ' price_amount ', 'price_currency_code']

for col in column_names:
  print('\n\n')
  print("==========================================================================")
  print(col.upper())
  print("VALUE COUNTS")
  print(acquisitions[col].value_counts())
  print("\n")

  print("DUPLICATE COUNTS")
  print(acquisitions.duplicated(subset=col, keep='first').sum())
  print("\n")

  print("NULL COUNTS AND PERCENTAGE")
  print(acquisitions[col].isnull().sum())
  print(acquisitions[col].isnull().sum()/len(acquisitions)*100)
  print('\n')
  
  print("NULL ROWS")
  display(acquisitions[acquisitions[col].isnull()])




COMPANY_PERMALINK
VALUE COUNTS
/organization/pando-networks                18
/organization/insieme                        5
/organization/travel-channel                 4
/organization/udcast                         4
/organization/webshots                       4
                                            ..
/organization/fortis-healthcare              1
/organization/fortress-solutions             1
/organization/fortress-technologies          1
/organization/fortsum-business-solutions     1
/organization/zystor                         1
Name: company_permalink, Length: 12790, dtype: int64


DUPLICATE COUNTS
279


NULL COUNTS AND PERCENTAGE
0
0.0


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,acquirer_permalink,acquirer_name,...,acquirer_country_code,acquirer_state_code,acquirer_region,acquirer_city,acquired_at,acquired_month,acquired_quarter,acquired_year,price_amount,price_currency_code





COMPANY_NAME
VALUE COUNTS
Pando Networks                18
Insieme                        5
Webshots                       4
UDcast                         4
Travel Channel                 4
                              ..
Fortis Healthcare              1
Fortress Solutions             1
Fortress Technologies          1
Fortsum Business Solutions     1
Zystor                         1
Name: company_name, Length: 12783, dtype: int64


DUPLICATE COUNTS
286


NULL COUNTS AND PERCENTAGE
0
0.0


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,acquirer_permalink,acquirer_name,...,acquirer_country_code,acquirer_state_code,acquirer_region,acquirer_city,acquired_at,acquired_month,acquired_quarter,acquired_year,price_amount,price_currency_code





COMPANY_CATEGORY_LIST
VALUE COUNTS
|Software|                                                                    1254
|Curated Web|                                                                  578
|Biotechnology|                                                                475
|Enterprise Software|                                                          351
|Mobile|                                                                       293
                                                                              ... 
|Internet|Identity|Media|Personalization|Finance|Fraud Detection|Software|       1
|Publishing|News|Media|Video|Advertising|                                        1
|Mobile|Crowdsourcing|Gas|Travel|Oil|Curated Web|                                1
|Media|Hardware|Computers|Hardware + Software|                                   1
|Design|Health and Wellness|Hardware + Software|                                 1
Name: company_category_list, Length: 3649, dtype:

...

,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,acquirer_permalink,acquirer_name,...,acquirer_country_code,acquirer_state_code,acquirer_region,acquirer_city,acquired_at,acquired_month,acquired_quarter,acquired_year,price_amount,price_currency_code
3093,/organization/datapad-inc,DataPad,|Big Data Analytics|Business Intelligence|Big ...,Analytics,USA,CA,SF Bay Area,San Francisco,/organization/cloudera,Cloudera,...,USA,CA,SF Bay Area,Palo Alto,0030-09-14,NaN,NaN,NaN,NaN,USD
11206,/organization/tastykhana,TastyKhana,|E-Commerce|,E-Commerce,IND,NaN,Pune,Pune,/organization/foodpanda,foodpanda / hellofood,...,DEU,NaN,Berlin,Berlin,0017-11-14,NaN,NaN,NaN,NaN,USD





 PRICE_AMOUNT 
VALUE COUNTS
1.000000e+07    56
3.000000e+07    52
1.000000e+08    51
5.000000e+07    49
2.000000e+07    47
                ..
5.460000e+08     1
2.260000e+07     1
1.170000e+09     1
5.240000e+07     1
6.200000e+04     1
Name:  price_amount , Length: 1311, dtype: int64


DUPLICATE COUNTS
11757


NULL COUNTS AND PERCENTAGE
9358
71.60456041013084


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,acquirer_permalink,acquirer_name,...,acquirer_country_code,acquirer_state_code,acquirer_region,acquirer_city,acquired_at,acquired_month,acquired_quarter,acquired_year,price_amount,price_currency_code
0,/organization/waywire,#waywire,|Entertainment|Politics|Social Media|News|,News,USA,NY,New York City,New York,/organization/magnify,Waywire Networks,...,USA,NY,New York City,New York,2013-10-17,2013-10,2013-Q4,2013.0,NaN,USD
1,/organization/fluff-friends,(fluff)Friends,NaN,NaN,NaN,NaN,NaN,NaN,/organization/social-gaming-network,SGN (Social Gaming Network),...,USA,CA,Los Angeles,Beverly Hills,2008-09-16,2008-09,2008-Q3,2008.0,NaN,USD
3,/organization/vandaele-holdings,.,NaN,NaN,NaN,NaN,NaN,NaN,/organization/hi7e,HI7E,...,USA,FL,Palm Beaches,West Palm Beach,2011-01-01,2011-01,2011-Q1,2011.0,NaN,USD
5,/organization/vantronix,.vantronix,|Cloud Security|Network Security|Security|,Cloud Security,DEU,NaN,Hanover,Hanover,/organization/compumatica,Compumatica,...,NaN,NaN,NaN,NaN,2011-01-01,2011-01,2011-Q1,2011.0,NaN,USD
6,/organization/003-ru,003.RU,NaN,NaN,RUS,NaN,Moscow,Moscow,/organization/media-saturn,Media Saturn,...,ESP,NaN,Barcelona,Barcelona,2012-07-23,2012-07,2012-Q3,2012.0,NaN,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13063,/organization/zynamics,zynamics,|Security|,Security,DEU,NaN,Bochum,Bochum,/organization/google,Google,...,USA,CA,SF Bay Area,Mountain View,2011-03-01,2011-03,2011-Q1,2011.0,NaN,USD
13064,/organization/zync,Zync,|Curated Web|,Curated Web,NaN,NaN,NaN,NaN,/organization/where-com,Where,...,USA,MA,Boston,Boston,2008-02-07,2008-02,2008-Q1,2008.0,NaN,USD
13065,/organization/zync-render,Zync Render,|Digital Media|,Digital Media,USA,MA,Boston,Boston,/organization/google,Google,...,USA,CA,SF Bay Area,Mountain View,2014-08-26,2014-08,2014-Q3,2014.0,NaN,USD
13066,/organization/zynk-mobile,Zynk Mobile,NaN,NaN,NaN,NaN,NaN,NaN,/organization/zenvia,Zenvia,...,BRA,NaN,Porto Alegre,Porto Alegre,2013-03-01,2013-03,2013-Q1,2013.0,NaN,USD





PRICE_CURRENCY_CODE
VALUE COUNTS
USD    12800
EUR      115
GBP      108
CAD       16
AUD       10
JPY        7
SEK        4
NZD        2
NOK        1
AZN        1
SAR        1
Name: price_currency_code, dtype: int64


DUPLICATE COUNTS
13057


NULL COUNTS AND PERCENTAGE
4
0.03060677940163746


NULL ROWS


,company_permalink,company_name,company_category_list,company_market,company_country_code,company_state_code,company_region,company_city,acquirer_permalink,acquirer_name,...,acquirer_country_code,acquirer_state_code,acquirer_region,acquirer_city,acquired_at,acquired_month,acquired_quarter,acquired_year,price_amount,price_currency_code
784,/organization/anywhere-fm,Anywhere.FM,|Music|,Music,NaN,NaN,NaN,NaN,/organization/imeem,imeem,...,USA,CA,SF Bay Area,San Francisco,2008-01-28,2008-01,2008-Q1,2008.0,NaN,NaN
6615,/organization/local2me,Local2Me,|Local|Advertising|,Local,USA,CA,SF Bay Area,San Mateo,/organization/smalltown,Smalltown,...,USA,CA,SF Bay Area,San Mateo,2007-12-01,2007-12,2007-Q4,2007.0,NaN,NaN
10504,/organization/snocap,Snocap,|Content|Social Network Media|Peer-to-Peer|Games|,Games,USA,CA,SF Bay Area,San Francisco,/organization/imeem,imeem,...,USA,CA,SF Bay Area,San Francisco,2008-02-01,2008-02,2008-Q1,2008.0,NaN,NaN
12842,/organization/xiaonei,Xiaonei,|Networking|Facebook Applications|Curated Web|,Facebook Applications,CHN,NaN,CHN - Other,NaN,/organization/renren-inc,Renren Inc.,...,CHN,NaN,CHN - Other,Chaoyang,2006-10-24,2006-10,2006-Q4,2006.0,NaN,NaN


#Additional Tests for Data Analysis and Entity Resolution Understanding

In [26]:
mergeI = pd.merge(companies[['permalink']], investments[['investor_permalink']], left_on='permalink', right_on='investor_permalink', how='outer', indicator=True)
mergeI = mergeI[mergeI['_merge'] == 'both']
mergeI


,permalink,investor_permalink,_merge
67,/organization/1c-company,/organization/1c-company,both
107,/organization/21vianet,/organization/21vianet,both
136,/organization/2b-angels,/organization/2b-angels,both
137,/organization/2b-angels,/organization/2b-angels,both
138,/organization/2b-angels,/organization/2b-angels,both
...,...,...,...
52606,/organization/zipcar,/organization/zipcar,both
52607,/organization/zipcar,/organization/zipcar,both
52717,/organization/zoominfo,/organization/zoominfo,both
52778,/organization/zulily,/organization/zulily,both


In [27]:
mergeA = pd.merge(companies[['permalink']], acquisitions[['acquirer_permalink']], left_on='permalink', right_on='acquirer_permalink', how='outer', indicator=True)
mergeA = mergeA[mergeA['_merge'] == 'both']
mergeA

,permalink,acquirer_permalink,_merge
38,/organization/11i-solutions,/organization/11i-solutions,both
39,/organization/11i-solutions,/organization/11i-solutions,both
68,/organization/1c-company,/organization/1c-company,both
91,/organization/1stdibs,/organization/1stdibs,both
94,/organization/1world-online,/organization/1world-online,both
...,...,...,...
52170,/organization/zynga,/organization/zynga,both
52171,/organization/zynga,/organization/zynga,both
52175,/organization/zyraz-technology,/organization/zyraz-technology,both
52182,/organization/x,/organization/x,both


In [28]:
mergeA = pd.merge(investments[['investor_permalink']], acquisitions[['acquirer_permalink']], left_on='investor_permalink', right_on='acquirer_permalink', how='outer', indicator=True)
mergeA = mergeA[mergeA['_merge'] != 'both']
mergeA

,investor_permalink,acquirer_permalink,_merge
0,/person/jamessss-bondddd,NaN,left_only
1,/organization/ekaf,NaN,left_only
2,/person/antonio-murroni,NaN,left_only
3,/person/filippo-murroni,NaN,left_only
4,/organization/crp-companhia-de-participacoes,NaN,left_only
...,...,...,...
172772,NaN,/organization/webjet-com-au,right_only
172773,NaN,/organization/nestle-purina-petcare,right_only
172774,NaN,/organization/cupcake-digital,right_only
172775,NaN,/organization/withings,right_only


In [29]:
agg = rounds.groupby('company_permalink').sum().reset_index(drop=False)[['company_permalink', ' raised_amount_usd ']]
agg = pd.merge(companies[['permalink', ' funding_total_usd ']], agg, left_on='permalink', right_on='company_permalink', how='outer', indicator=True)
agg

check = agg[agg['_merge'] != 'both']
print(len(check))
check

def replace_null(x):
  if x == 0.0:
    return np.nan
  else:
    return x
# found out that prysm is duplicate
both = agg[agg['_merge'] == 'both']
#both[' raised_amount_usd '] = both[' raised_amount_usd '].apply(lambda x: replace_null(x))
both = both[both[' funding_total_usd '] != both[' raised_amount_usd ']]
both = both.sort_values(by=' raised_amount_usd ', ascending=False)
both = both[both[' raised_amount_usd '] != 0.0]
both

35


,permalink,funding_total_usd,company_permalink,raised_amount_usd,_merge
34074,/organization/pure-storage,534856468.0,/organization/pure-storage,474856468.0,both
33939,/organization/prysm,145521193.0,/organization/prysm,293080123.0,both
33940,/organization/prysm,147558930.0,/organization/prysm,293080123.0,both
31086,/organization/oxford-nanopore-technologies,270725149.0,/organization/oxford-nanopore-technologies,211725149.0,both
40911,/organization/stripe,120000000.0,/organization/stripe,190000000.0,both
...,...,...,...,...,...
29330,/organization/niwa,200000.0,/organization/niwa,150000.0,both
37387,/organization/seesearch,27181.0,/organization/seesearch,54362.0,both
45238,/organization/urbita,675000.0,/organization/urbita,25000.0,both
4228,/organization/babbaco,4525000.0,/organization/babbaco,25000.0,both


#Creation of Tables according to our Schema

## Plan for cleaning

1.   Preliminary cleaning of companies, investments, rounds 
and acquisitions.
2.   Next, using the companies table, I create a company_entity for the financial_entity
3. I create investor_entity using the investments table
4. I create the acquirer entity from the acquisitions table
5. Original Rounds table contains after being merged on funding_round_permalink has three cases (post checking with original companies) - 
  *   companies in round but not in original companies - need to add to company_entity 
  *   Companies in original but not in rounds - nothing to be done
  *   investor as null for the round post merging - needs to be imputed to unknown in both the final rounds table and investor_entity
6. Acquisitions after checking merge with original companies has companies that are in acquisitions table that are not in original companies - needs to be added to company_entity

7. Finally need to combine all three (and update is_a accordingly if an entity is more than one of the three)

8. Give them a unique integer ID




##Cleaning up Companies

###Fixing category_list

In [30]:
def reformat_lists(x):
  if x is np.nan:
    return 'unknown'
  x = str(x)
  x = x.replace("|","   ")
  x = x.strip()
  x = x.replace("   ",", ")
  return x

companies['category_list'] = companies['category_list'].apply(lambda x: reformat_lists(x))

###Fixing Permalink duplicates

In [31]:
companies.at[33940, 'funding_rounds'] = 4
companies.at[33940, 'last_funding_at'] = '2014-11-25'
companies.at[33940, ' funding_total_usd '] = 100000000.0 + 44808930.0 + 2750000.0 + 145521193
companies[companies.duplicated(subset='permalink', keep='first')]

companies.at[44033, 'funding_rounds'] = 5
companies.at[44033, 'last_funding_at'] = '2014-07-18'
companies.at[44033, ' funding_total_usd '] = 286873.0 + 45321.0

companies = companies.drop(33939)
companies = companies.drop(44034)



In [32]:
companies = companies.reset_index(drop=False)

###Filling name null

In [33]:
companies.at[28221, 'name'] = 'Tell-IT'

###Filling Status nulls

In [34]:
def replace_nan_status(x):
  if x is np.nan:
    return 'unknown'
  else:
    return x

companies['status'] = companies['status'].apply(lambda x: replace_nan_status(x))

###Changing date to Datetime dtype and formatted

In [35]:
companies['founded_at'] = pd.to_datetime(companies['founded_at'],format='%Y-%m-%d', errors = 'coerce')

##Cleaning Investments

###Fixing investor_category_list

In [36]:
investments['investor_category_list'] = investments['investor_category_list'].apply(lambda x: reformat_lists(x))

###Resolving null investor_permalink

66 investors were imputed to as unknowns because they were nulls, and investor permalink cannot be null

In [37]:
nulls = investments[investments['investor_permalink'].isna()]
index = nulls.index

investor_impute_count = 1
for i in index:
  investments.at[i, 'investor_permalink'] = 'unknown' + str(investor_impute_count)
  investments.at[i, 'investor_name'] = 'unknown'
  investor_impute_count+=1



###Creating isPerson column

In [38]:
def create_is_person(x):
  if 'organization' in x:
    return "False"
  elif 'person' in x:
    return "True"
  else:
    return np.nan

investments['is_person'] = investments['investor_permalink'].apply(lambda x: create_is_person(x))

##Cleaning Acquisitions

###Fixing acquirer_category_list

In [39]:
acquisitions['acquirer_category_list'] = acquisitions['acquirer_category_list'].apply(lambda x: reformat_lists(x))

###Fixing null price_currency_code

In [40]:
nulls = acquisitions[acquisitions['price_currency_code'].isna()]
index = nulls.index

for i in index:
  acquisitions.at[i, 'price_currency_code'] = 'USD'

acquisitions.at[12842, 'price_currency_code'] = 'CNY'

###Converting acquired_at to datetime

In [41]:
acquisitions['acquired_at'] = pd.to_datetime(acquisitions['acquired_at'],format='%Y-%m-%d', errors = 'coerce')

##Cleaning Rounds

###Converting funded_at to datetime

In [42]:
rounds['funded_at'] = pd.to_datetime(rounds['funded_at'],format='%Y-%m-%d', errors = 'coerce')

###Creating round_number

In [43]:
rounds['round_number'] = np.arange(len(rounds))

## Creating Financial_Entity Components

###Restructuring companies

In [44]:
company_entity = companies[['permalink', 'name', 'category_list', ' market ',  'country_code', 'state_code', 'city']]
company_entity.insert(7, 'is_a', 'company')
company_entity = company_entity.rename(columns={'category_list': 'categories', ' market ': 'market', 'country_code': 'country', 'state_code': 'state'})


company_entity 

,permalink,name,categories,market,country,state,city,is_a
0,/organization/waywire,#waywire,"Entertainment, Politics, Social Media, News",News,USA,NY,New York,company
1,/organization/tv-communications,&TV Communications,Games,Games,USA,CA,Los Angeles,company
2,/organization/rock-your-paper,'Rock' Your Paper,"Publishing, Education",Publishing,EST,NaN,Tallinn,company
3,/organization/in-touch-network,(In)Touch Network,"Electronics, Guides, Coffee, Restaurants, Musi...",Electronics,GBR,NaN,London,company
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,"Tourism, Entertainment, Games",Tourism,USA,TX,Fort Worth,company
...,...,...,...,...,...,...,...,...
49431,/organization/zzish,Zzish,"Analytics, Gamification, Developer APIs, iOS, ...",Education,GBR,NaN,London,company
49432,/organization/zznode-science-and-technology-co...,ZZNode Science and Technology,Enterprise Software,Enterprise Software,CHN,NaN,Beijing,company
49433,/organization/zzzzapp-com,Zzzzapp Wireless ltd.,"Web Development, Advertising, Wireless, Mobile",Web Development,HRV,NaN,Split,company
49434,/organization/a-list-games,[a]list games,Games,Games,NaN,NaN,NaN,company


###Restructuring Investments

In [45]:
investor_entity = investments[['investor_permalink', 'investor_name', 'investor_category_list', 'investor_market',  'investor_country_code', 'investor_state_code', 'investor_city']]

#To see which were duplicated
#investor_entity[investor_entity.duplicated(subset='investor_permalink', keep='first')]

investor_entity = investor_entity.drop_duplicates("investor_permalink")
investor_entity.insert(7, 'is_a', 'investor')
investor_entity = investor_entity.rename(columns={'investor_permalink': 'permalink', 'investor_name': 'name','investor_category_list': 'categories', 'investor_market': 'market', 'investor_country_code': 'country', 'investor_state_code': 'state', 'investor_city': 'city'})
investor_entity

,permalink,name,categories,market,country,state,city,is_a
0,/person/jamessss-bondddd,jamessss bondddd,unknown,NaN,NaN,NaN,NaN,investor
1,/organization/ekaf,Ekaf,unknown,NaN,NaN,NaN,NaN,investor
2,/person/antonio-murroni,ANTONIO MURRONI,unknown,NaN,NaN,NaN,NaN,investor
3,/person/filippo-murroni,FILIPPO Murroni,unknown,NaN,NaN,NaN,NaN,investor
4,/organization/crp-companhia-de-participacoes,CRP Companhia de Participações,unknown,NaN,NaN,NaN,NaN,investor
...,...,...,...,...,...,...,...,...
114446,/person/doug-cobb,Doug Cobb,unknown,NaN,NaN,NaN,NaN,investor
114452,/person/s-kris-gopalakrishnan,"S. ""Kris"" Gopalakrishnan",unknown,NaN,NaN,NaN,NaN,investor
114466,/organization/dcl-ventures-inc,"DCL Ventures, Inc.",Pets,Pets,USA,NY,New York,investor
114482,/organization/math-venture-partners,Math Venture Partners,unknown,NaN,USA,IL,Chicago,investor


###Restructuring Acquisitions

In [46]:
acquirer_entity = acquisitions[['acquirer_permalink', 'acquirer_name', 'acquirer_category_list', 'acquirer_market',  'acquirer_country_code', 'acquirer_state_code', 'acquirer_city']]

#To see which were duplicated
#acquirer_entity[acquirer_entity.duplicated(subset='acquirer_permalink', keep='first')]

acquirer_entity.insert(7, 'is_a', 'acquirer')
acquirer_entity = acquirer_entity.drop_duplicates("acquirer_permalink")


acquirer_entity = acquirer_entity.rename(columns={'acquirer_permalink': 'permalink', 'acquirer_name': 'name','acquirer_category_list': 'categories', 'acquirer_market': 'market', 'acquirer_country_code': 'country', 'acquirer_state_code': 'state', 'acquirer_city': 'city'})
acquirer_entity

,permalink,name,categories,market,country,state,city,is_a
0,/organization/magnify,Waywire Networks,"Curated Web, Video",Curated Web,USA,NY,New York,acquirer
1,/organization/social-gaming-network,SGN (Social Gaming Network),Games,Games,USA,CA,Beverly Hills,acquirer
2,/organization/nationstar-mortgage-holdings,Nationstar Mortgage Holdings,Finance,Finance,USA,TX,Lewisville,acquirer
3,/organization/hi7e,HI7E,unknown,NaN,USA,FL,West Palm Beach,acquirer
4,/organization/neustar,Neustar,"Telecommunications, Analytics, Information Tec...",Analytics,USA,VA,Sterling,acquirer
...,...,...,...,...,...,...,...,...
13052,/organization/webjet-com-au,Webjet.com.au,"Travel & Tourism, Hotels, Travel",Travel & Tourism,AUS,NaN,Melbourne,acquirer
13053,/organization/nestle-purina-petcare,Nestle Purina Petcare,Pets,Pets,USA,MO,St Louis,acquirer
13056,/organization/cupcake-digital,Cupcake Digital,Games,Games,USA,NY,New York,acquirer
13060,/organization/withings,Withings,Hardware + Software,Hardware + Software,FRA,NaN,Issy-les-moulineaux,acquirer


##Round

In [47]:
round_schema = pd.merge(rounds[['company_permalink','funding_round_permalink', 'funding_round_type', ' raised_amount_usd ', 'funded_at', 'round_number']], investments[['investor_permalink','funding_round_permalink']], left_on='funding_round_permalink', right_on='funding_round_permalink', how='outer', indicator=True)

###Fixing Investor Nulls

Imputing investor permalinks in cases where a funding round does not have a investor associated with it --> setting investor to be unknown. Adding newly created values to investor entity as well.

In [48]:
# Get a list of all funding_round_permalinks that have this problem
problematic_funding_rounds = round_schema[round_schema['_merge'] == 'left_only'].index
print(problematic_funding_rounds)
count = 67
for i in problematic_funding_rounds:
  round_schema.at[i, 'investor_permalink'] = 'unknown' + str(count)
  investor_entity = investor_entity.append({'permalink': ('unknown' + str(count)), 'name': 'unknown', 'is_a': 'investor' }, ignore_index=True)
  count += 1



round_schema[round_schema['_merge'] == 'left_only']


Int64Index([     6,      7,      9,     10,     11,     12,     18,     21,
                22,     23,
            ...
            148061, 148062, 148063, 148064, 148068, 148074, 148075, 148081,
            148082, 148095],
           dtype='int64', length=33590)


,company_permalink,funding_round_permalink,funding_round_type,raised_amount_usd,funded_at,round_number,investor_permalink,_merge
6,/organization/tv-communications,/funding-round/59a3669a64e39360c2b939300bcda162,venture,3000000.0,2010-09-23,1,unknown67,left_only
7,/organization/tv-communications,/funding-round/86d22afc65107b6941e6c43c671ecbb8,venture,1000000.0,2010-06-04,2,unknown68,left_only
9,/organization/in-touch-network,/funding-round/33c3f135f05d7b734b8d7b7c8ae82647,seed,1500000.0,2011-04-01,4,unknown69,left_only
10,/organization/r-ranch-and-mine,/funding-round/029720f7eeb218f51c43df5155671472,equity_crowdfunding,10000.0,2014-08-17,5,unknown70,left_only
11,/organization/r-ranch-and-mine,/funding-round/766b0bcead9ca8560af5e1ade579fb7f,equity_crowdfunding,50000.0,2014-09-26,6,unknown71,left_only
...,...,...,...,...,...,...,...,...
148074,/organization/zyrra,/funding-round/cc8cd96a500332d5b4f4bcbddb306099,angel,875000.0,2010-12-15,83854,unknown33652,left_only
148075,/organization/zyrra,/funding-round/7a0fa791c96c9c447ee54662395beab9,venture,140500.0,2012-10-18,83855,unknown33653,left_only
148081,/organization/zzzzapp-com,/funding-round/87b5f54b90a09a05ed3d1b8ee509ed11,convertible_note,12918.0,2014-09-10,83860,unknown33654,left_only
148082,/organization/zzzzapp-com,/funding-round/22ef2fafb4d20ac3aa4b86143dbf6c8e,seed,7000.0,2011-11-01,83861,unknown33655,left_only


Checking to see if finanicial entity was correctly updated

In [49]:
investor_entity[investor_entity['permalink'].str.contains('unknown')]

,permalink,name,categories,market,country,state,city,is_a
312,unknown1,unknown,unknown,NaN,NaN,NaN,NaN,investor
3388,unknown2,unknown,unknown,NaN,NaN,NaN,NaN,investor
8336,unknown3,unknown,unknown,NaN,NaN,NaN,NaN,investor
8952,unknown4,unknown,unknown,NaN,NaN,NaN,NaN,investor
9268,unknown5,unknown,unknown,NaN,NaN,NaN,NaN,investor
...,...,...,...,...,...,...,...,...
55927,unknown33652,unknown,NaN,NaN,NaN,NaN,NaN,investor
55928,unknown33653,unknown,NaN,NaN,NaN,NaN,NaN,investor
55929,unknown33654,unknown,NaN,NaN,NaN,NaN,NaN,investor
55930,unknown33655,unknown,NaN,NaN,NaN,NaN,NaN,investor


###Adding companies from Rounds

There are certain companies found in Rounds table that were not present in the original companies table. I add these to the company entity.

In [50]:
mergeCR = pd.merge(rounds, company_entity[['permalink']], left_on='company_permalink', right_on='permalink', how='outer', indicator=True)
missing_companies = mergeCR[mergeCR['_merge'] == 'left_only'][['company_permalink', 'company_name', 'company_category_list', 'company_market',  'company_country_code', 'company_state_code', 'company_city']].drop_duplicates()
missing_companies.insert(7, 'is_a', 'company')
missing_companies = missing_companies.rename(columns={'company_permalink': 'permalink', 'company_name': 'name','company_category_list': 'categories', 'company_market': 'market', 'company_country_code': 'country', 'company_state_code': 'state', 'company_city': 'city'})

company_entity = pd.concat([company_entity, missing_companies], axis=0)
company_entity

,permalink,name,categories,market,country,state,city,is_a
0,/organization/waywire,#waywire,"Entertainment, Politics, Social Media, News",News,USA,NY,New York,company
1,/organization/tv-communications,&TV Communications,Games,Games,USA,CA,Los Angeles,company
2,/organization/rock-your-paper,'Rock' Your Paper,"Publishing, Education",Publishing,EST,NaN,Tallinn,company
3,/organization/in-touch-network,(In)Touch Network,"Electronics, Guides, Coffee, Restaurants, Musi...",Electronics,GBR,NaN,London,company
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,"Tourism, Entertainment, Games",Tourism,USA,TX,Fort Worth,company
...,...,...,...,...,...,...,...,...
64465,/organization/shift-payments,Shift Payments,|Finance Technology|Personal Finance|Finance|,Finance,USA,CA,San Francisco,company
67874,/organization/sparkow,Sparkow,NaN,NaN,NaN,NaN,NaN,company
74118,/organization/totems,TOTEMS,NaN,NaN,NaN,NaN,NaN,company
80113,/organization/wedpics,WedPics,NaN,NaN,NaN,NaN,NaN,company


##Adding companies from acquisitions

In [51]:
mergeCA = pd.merge(acquisitions, company_entity[['permalink']], left_on='company_permalink', right_on='permalink', how='outer', indicator=True)
missing_companiesA = mergeCA[mergeCA['_merge'] == 'left_only'][['company_permalink', 'company_name', 'company_category_list', 'company_market',  'company_country_code', 'company_state_code', 'company_city']].drop_duplicates()
missing_companiesA.insert(7, 'is_a', 'company')
missing_companiesA = missing_companiesA.rename(columns={'company_permalink': 'permalink', 'company_name': 'name','company_category_list': 'categories', 'company_market': 'market', 'company_country_code': 'country', 'company_state_code': 'state', 'company_city': 'city'})

company_entity = pd.concat([company_entity, missing_companiesA], axis=0)

##Combining all three entities to form Financial Entity

####Starting with merging Company and Investor

In [52]:
commonCI = pd.merge(company_entity[['permalink']], investor_entity[['permalink']], left_on='permalink', right_on='permalink', how='outer', indicator=True)
commonCIBoth = commonCI[commonCI['_merge'] == 'both']
commonCIBoth = commonCIBoth['permalink']
commonCIRight = commonCI[commonCI['_merge'] == 'right_only']
commonCIRight = commonCIRight['permalink']

financial_entity_schema = company_entity.copy()

for i in commonCIBoth:
  financial_entity_schema.loc[financial_entity_schema['permalink'] == i, 'is_a'] = financial_entity_schema.loc[financial_entity_schema['permalink'] == i, 'is_a'] + ', investor'

for i in commonCIRight:
  financial_entity_schema = financial_entity_schema.append(investor_entity[investor_entity['permalink'] == i], ignore_index=True)


financial_entity_schema


,permalink,name,categories,market,country,state,city,is_a
0,/organization/waywire,#waywire,"Entertainment, Politics, Social Media, News",News,USA,NY,New York,company
1,/organization/tv-communications,&TV Communications,Games,Games,USA,CA,Los Angeles,company
2,/organization/rock-your-paper,'Rock' Your Paper,"Publishing, Education",Publishing,EST,NaN,Tallinn,company
3,/organization/in-touch-network,(In)Touch Network,"Electronics, Guides, Coffee, Restaurants, Musi...",Electronics,GBR,NaN,London,company
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,"Tourism, Entertainment, Games",Tourism,USA,TX,Fort Worth,company
...,...,...,...,...,...,...,...,...
113716,unknown33652,unknown,NaN,NaN,NaN,NaN,NaN,investor
113717,unknown33653,unknown,NaN,NaN,NaN,NaN,NaN,investor
113718,unknown33654,unknown,NaN,NaN,NaN,NaN,NaN,investor
113719,unknown33655,unknown,NaN,NaN,NaN,NaN,NaN,investor


In [53]:
#financial_entity_schema[financial_entity_schema['is_a'] == 'investor']
financial_entity_schema[financial_entity_schema.duplicated(subset='permalink', keep='first')]

,permalink,name,categories,market,country,state,city,is_a


####Merging Result with Acquirers

In [54]:
commonCA = pd.merge(financial_entity_schema[['permalink']], acquirer_entity[['permalink']], left_on='permalink', right_on='permalink', how='outer', indicator=True)
commonCABoth = commonCA[commonCA['_merge'] == 'both']
commonCABoth = commonCABoth['permalink']
commonCARight = commonCA[commonCA['_merge'] == 'right_only']
commonCARight = commonCARight['permalink']

for i in commonCABoth:
  financial_entity_schema.loc[financial_entity_schema['permalink'] == i, 'is_a'] = financial_entity_schema.loc[financial_entity_schema['permalink'] == i, 'is_a'] + ', acquirer'

for i in commonCARight:
  financial_entity_schema = financial_entity_schema.append(acquirer_entity[acquirer_entity['permalink'] == i], ignore_index=True)


financial_entity_schema

,permalink,name,categories,market,country,state,city,is_a
0,/organization/waywire,#waywire,"Entertainment, Politics, Social Media, News",News,USA,NY,New York,company
1,/organization/tv-communications,&TV Communications,Games,Games,USA,CA,Los Angeles,company
2,/organization/rock-your-paper,'Rock' Your Paper,"Publishing, Education",Publishing,EST,NaN,Tallinn,company
3,/organization/in-touch-network,(In)Touch Network,"Electronics, Guides, Coffee, Restaurants, Musi...",Electronics,GBR,NaN,London,company
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,"Tourism, Entertainment, Games",Tourism,USA,TX,Fort Worth,company
...,...,...,...,...,...,...,...,...
117200,/organization/apsis-sweden,Apsis Sweden,Advertising,Advertising,SWE,NaN,Malma,acquirer
117201,/organization/webjet-com-au,Webjet.com.au,"Travel & Tourism, Hotels, Travel",Travel & Tourism,AUS,NaN,Melbourne,acquirer
117202,/organization/nestle-purina-petcare,Nestle Purina Petcare,Pets,Pets,USA,MO,St Louis,acquirer
117203,/organization/cupcake-digital,Cupcake Digital,Games,Games,USA,NY,New York,acquirer


In [55]:
#financial_entity_schema[financial_entity_schema['is_a'] == 'company, investor, acquirer']
financial_entity_schema[financial_entity_schema.duplicated(subset='permalink', keep='first')]

,permalink,name,categories,market,country,state,city,is_a


###Creating ID

In [56]:
financial_entity_schema['ID'] = np.arange(len(financial_entity_schema))
financial_entity_schema

,permalink,name,categories,market,country,state,city,is_a,ID
0,/organization/waywire,#waywire,"Entertainment, Politics, Social Media, News",News,USA,NY,New York,company,0
1,/organization/tv-communications,&TV Communications,Games,Games,USA,CA,Los Angeles,company,1
2,/organization/rock-your-paper,'Rock' Your Paper,"Publishing, Education",Publishing,EST,NaN,Tallinn,company,2
3,/organization/in-touch-network,(In)Touch Network,"Electronics, Guides, Coffee, Restaurants, Musi...",Electronics,GBR,NaN,London,company,3
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,"Tourism, Entertainment, Games",Tourism,USA,TX,Fort Worth,company,4
...,...,...,...,...,...,...,...,...,...
117200,/organization/apsis-sweden,Apsis Sweden,Advertising,Advertising,SWE,NaN,Malma,acquirer,117200
117201,/organization/webjet-com-au,Webjet.com.au,"Travel & Tourism, Hotels, Travel",Travel & Tourism,AUS,NaN,Melbourne,acquirer,117201
117202,/organization/nestle-purina-petcare,Nestle Purina Petcare,Pets,Pets,USA,MO,St Louis,acquirer,117202
117203,/organization/cupcake-digital,Cupcake Digital,Games,Games,USA,NY,New York,acquirer,117203


##Company

In [57]:
company_schema = financial_entity_schema[financial_entity_schema['is_a'].str.contains('company')][['permalink', 'ID']]
company_schema = pd.merge(company_schema, companies[['permalink', 'founded_at', 'homepage_url', 'status']], left_on='permalink', right_on='permalink', how='left')
company_schema = company_schema.sort_values(by='ID', ascending=True)
company_schema = company_schema[['ID', 'founded_at', 'homepage_url', 'status']]
company_schema = company_schema.rename(columns={'ID': 'company_ID', 'founded_at': 'founding_date', 'homepage_url': 'homepage_URL'})

In [58]:
company_schema['status'] = company_schema['status'].apply(lambda x: replace_nan_status(x))
company_schema

,company_ID,founding_date,homepage_URL,status
0,0,2012-06-01,http://www.waywire.com,acquired
1,1,NaT,http://enjoyandtv.com,operating
2,2,2012-10-26,http://www.rockyourpaper.org,operating
3,3,2011-04-01,http://www.InTouchNetwork.com,operating
4,4,2014-01-01,NaN,operating
...,...,...,...,...
58447,58447,NaT,NaN,unknown
58448,58448,NaT,NaN,unknown
58449,58449,NaT,NaN,unknown
58450,58450,NaT,NaN,unknown


##Investor

In [59]:
investor_schema = financial_entity_schema[financial_entity_schema['is_a'].str.contains('investor')][['permalink', 'ID']]
investor_schema = pd.merge(investor_schema, investments[['investor_permalink', 'is_person']], left_on='permalink', right_on='investor_permalink', how='left')
investor_schema = investor_schema.sort_values(by='ID', ascending=True)
investor_schema = investor_schema[['ID', 'is_person']]
investor_schema = investor_schema.rename(columns={'ID': 'investor_ID'})
investor_schema = investor_schema.drop_duplicates("investor_ID")
investor_schema

,investor_ID,is_person
0,67,False
1,107,False
13,136,False
14,185,False
15,190,False
...,...,...
148091,113716,NaN
148092,113717,NaN
148093,113718,NaN
148094,113719,NaN


##Acquisition

In [60]:
acquisition_schema = financial_entity_schema[financial_entity_schema['is_a'].str.contains('acquirer')][['permalink', 'ID']]
acquisition_schema = pd.merge(acquisition_schema, acquisitions[['acquirer_permalink', 'company_permalink', 'acquired_at', ' price_amount ', 'price_currency_code']], left_on='permalink', right_on='acquirer_permalink', how='left')
acquisition_schema = acquisition_schema.rename(columns={'ID': 'acquirer_ID', ' price_amount ': 'price', 'price_currency_code': 'currency', 'acquired_at': 'date'})
acquisition_schema = pd.merge(acquisition_schema, financial_entity_schema[['permalink', 'ID']], left_on='company_permalink', right_on='permalink', how='left')
acquisition_schema = acquisition_schema[['acquirer_ID', 'ID', 'date', 'price', 'currency']]
acquisition_schema = acquisition_schema.rename(columns={'ID': 'acquired_ID'})
acquisition_schema

,acquirer_ID,acquired_ID,date,price,currency
0,38,53372,2011-03-28,NaN,USD
1,38,54909,2011-04-05,NaN,USD
2,67,6603,2008-07-18,80000000.0,USD
3,90,55198,2012-09-18,NaN,USD
4,93,15577,2014-09-15,NaN,USD
...,...,...,...,...,...
13064,117200,49304,2012-06-17,NaN,USD
13065,117201,58440,2013-03-01,25000000.0,AUD
13066,117202,58441,2014-01-15,NaN,USD
13067,117203,49393,2014-03-12,NaN,USD


##Round Creation

In [61]:
round_schema = pd.merge(round_schema, financial_entity_schema[['permalink','ID']], left_on='company_permalink', right_on='permalink', how='left')
round_schema = round_schema.rename(columns={'ID': 'company_ID', 'funding_round_type': 'round_type', ' raised_amount_usd ': 'amount_USD', 'funded_at': 'date'})
round_schema = pd.merge(round_schema, financial_entity_schema[['permalink','ID']], left_on='investor_permalink', right_on='permalink', how='left')
round_schema = round_schema[['company_ID', 'ID', 'round_number', 'round_type', 'amount_USD', 'date']]

In [62]:
round_schema = round_schema.rename(columns={'ID': 'investor_ID'})
round_schema

,company_ID,investor_ID,round_number,round_type,amount_USD,date
0,0,66814,0,seed,1750000.0,2012-06-30
1,0,59532,0,seed,1750000.0,2012-06-30
2,0,65705,0,seed,1750000.0,2012-06-30
3,0,64316,0,seed,1750000.0,2012-06-30
4,0,71038,0,seed,1750000.0,2012-06-30
...,...,...,...,...,...,...
148091,49435,59224,83868,venture,10000000.0,2011-01-03
148092,49435,59037,83868,venture,10000000.0,2011-01-03
148093,49435,60929,83868,venture,10000000.0,2011-01-03
148094,49435,58520,83868,venture,10000000.0,2011-01-03


##Final Financial Entity

In [63]:
financial_entity_schema = financial_entity_schema[['ID', 'name', 'categories', 'market', 'country', 'state', 'city', 'is_a']]
financial_entity_schema

,ID,name,categories,market,country,state,city,is_a
0,0,#waywire,"Entertainment, Politics, Social Media, News",News,USA,NY,New York,company
1,1,&TV Communications,Games,Games,USA,CA,Los Angeles,company
2,2,'Rock' Your Paper,"Publishing, Education",Publishing,EST,NaN,Tallinn,company
3,3,(In)Touch Network,"Electronics, Guides, Coffee, Restaurants, Musi...",Electronics,GBR,NaN,London,company
4,4,-R- Ranch and Mine,"Tourism, Entertainment, Games",Tourism,USA,TX,Fort Worth,company
...,...,...,...,...,...,...,...,...
117200,117200,Apsis Sweden,Advertising,Advertising,SWE,NaN,Malma,acquirer
117201,117201,Webjet.com.au,"Travel & Tourism, Hotels, Travel",Travel & Tourism,AUS,NaN,Melbourne,acquirer
117202,117202,Nestle Purina Petcare,Pets,Pets,USA,MO,St Louis,acquirer
117203,117203,Cupcake Digital,Games,Games,USA,NY,New York,acquirer


#Convert to CSV files

In [65]:
financial_entity_schema.to_csv('Financial_Entity_Schema.csv', index=False)

In [66]:
company_schema.to_csv('Company_Schema.csv', index=False)

In [67]:
investor_schema.to_csv('Investor_Schema.csv', index=False)

In [68]:
round_schema.to_csv('Round_Schema.csv', index=False)

In [69]:
acquisition_schema.to_csv('Acquisition_Schema.csv', index=False)